In [1]:
import sys
import numpy as np
import pandas as pd
import torch

sys.path.append("../Graphormer")
from graphormer.modules import GraphormerGraphEncoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

/home/ww6p9/anaconda3/envs/DeepPurpose/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data

In [2]:
import pyximport
pyximport.install(setup_args={"include_dirs": np.get_include()})
from graphormer.data import algos
import argparse
from torch.utils.data import DataLoader, Dataset

In [3]:
N = 100
adj = torch.zeros([N, N], dtype=torch.bool)
adj = adj.to(torch.int)

In [4]:
def preprocess_item(adj):
    
    N = len(adj)
    x = torch.eye(N, dtype=torch.long)
    attn_edge_type = torch.zeros([N, N, 1], dtype=torch.long)
    shortest_path_result, path = algos.floyd_warshall(adj.numpy())
    max_dist = np.amax(shortest_path_result)
    edge_input = algos.gen_edge_input(max_dist, path, attn_edge_type.numpy())
    spatial_pos = torch.from_numpy((shortest_path_result)).long()
    attn_bias = torch.zeros([N + 1, N + 1], dtype=torch.float)  # with graph token

    item = {}
    item["x"] = x
    item["attn_bias"] = attn_bias
    item["attn_edge_type"] = attn_edge_type
    item["spatial_pos"] = spatial_pos
    item["in_degree"] = adj.long().sum(dim=1).view(-1)
    item["out_degree"] = item["in_degree"] # for undirected graph
    item["edge_input"] = torch.from_numpy(edge_input).long()
    
    return item

In [5]:
item = preprocess_item(adj)

In [6]:
class TrainDataset(Dataset):
    def __init__(self, adjs):
        self.adjs = adjs
    
    def __len__(self):
        return adjs.shape[2]
    
    def __getitem__(self, idx):
        return preprocess_item(adjs[:,:,idx])

In [7]:
adjs = torch.rand((N, N, 256))
adjs = adjs > 0.9
adjs = adjs.to(torch.int)

subnetwork_adj = np.load("subnetwork_adj.npy")
adjs = np.stack((subnetwork_adj,) * 256, axis=-1)
adjs = torch.from_numpy(adjs).to(torch.int)

train_dataset = TrainDataset(adjs)

In [8]:
train_loader = DataLoader(train_dataset,
                          batch_size=2,
                          shuffle=True,
                          num_workers=0,
                          pin_memory=False,
                          drop_last=True)

In [9]:
for step, inputs in enumerate(train_loader):
    break

In [10]:
inputs["x"].shape

torch.Size([2, 475, 475])

In [11]:
inputs["spatial_pos"].shape

torch.Size([2, 475, 475])

### Model

In [12]:
encoder = GraphormerGraphEncoder(
    num_atoms=1, # number of atom types in the graph
    num_in_degree=512, # number of in degree types in the graph
    num_out_degree=512, # number of out degree types in the graph
    num_edges=1, # number of edge types in the graph
    num_spatial=512, # number of spatial types in the graph
    num_edge_dis=1, # number of edge dis types in the graph
    edge_type="x", # edge type in the graph
    multi_hop_max_dist=512, # max number of edges considered in the edge encoding
)

# encoder

In [13]:
encoder = encoder.to(device)

In [14]:
for k, v in inputs.items():
    print(k)
    inputs[k] = v.to(device)

x
attn_bias
attn_edge_type
spatial_pos
in_degree
out_degree
edge_input


In [15]:
_, graph_rep = encoder(inputs)

In [16]:
graph_rep.shape

torch.Size([2, 768])

In [17]:
print(graph_rep)

tensor([[ 0.8468,  0.5050,  0.4485,  ...,  2.3219, -1.4351,  2.2487],
        [-0.0182,  1.1045, -0.3592,  ...,  0.5575,  0.1404,  0.4228]],
       device='cuda:0', grad_fn=<SliceBackward>)


In [18]:
graph_rep

tensor([[ 0.8468,  0.5050,  0.4485,  ...,  2.3219, -1.4351,  2.2487],
        [-0.0182,  1.1045, -0.3592,  ...,  0.5575,  0.1404,  0.4228]],
       device='cuda:0', grad_fn=<SliceBackward>)